In [1]:
import time, subprocess, os
import concurrent.futures
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.common.exceptions import WebDriverException, NoSuchElementException
from utils import clear_sessions, check_and_run_docker, get_proxies, check_container

In [3]:
check_and_run_docker()
time.sleep(10)
clear_sessions()

Docker container not found, running...
6afcd7d6fa46a8c71609941fcff85371aaad7e7f8fe1892af09bde5a3688c52d
Docker container running


In [4]:
# country_css = "div.firm-container > div.firm-services > div.firm-location.custom_tooltip > span::text"
# country = None
# company_block_css = "#service-providers > div > div.directory-list > ul > li.firm-wrapper"
# company_name_css = "div.firm-header > div > h3 > a::text"
# website_link_css = "div.firm-header > a::attr(href)"
# next_page_css = "#service-providers > div > div.directory-list > div.pagination-wrapper > ul > li.next-page > a"
# listing_website_start_url = "https://www.goodfirms.co/it-services/australia"
# page_num = 161
# chromedriver_command_executor = "http://localhost:4444/wd/hub"
# folder_name = "goodfirms_au"
# is_next_page_url_in_format = False
# category = "IT Services"

In [5]:
category_list_css = "#site-header-main > nav > ul > li.menu-item.menu-drop.agency-drop > div > div.menu-drop--sub_main.agency-drop--main > div.menu-drop--sub_category > ul > li > a"
listing_website = "https://www.designrush.com/agency/digital-agencies"
chromedriver_command_executor = "http://localhost:4444/wd/hub"
def get_categories_with_links(category_list_css,listing_website, chromedriver_command_executor):
    options = Options()
    driver = None
    try:
        signal = True
        options.add_argument('--ignore-certificate-errors')
        options.add_argument('--ignore-ssl-errors=yes')
        options.add_argument('--no-sandbox')
        options.add_argument('--disable-dev-shm-usage')
        # options.add_argument("--headless")

        # Set the proxy
        # random_proxy = random.choice(PROXIES)
        # print("Proxy in use: %s" % random_proxy)
        # options.add_argument('--proxy-server=%s' % random_proxy)
        # options.add_argument('--proxy-server=%s' % '104.234.157.100:21278')

        driver = webdriver.Remote(
            command_executor=chromedriver_command_executor,
            options=options,
        )
        driver.get(listing_website)
        category_blocks = driver.find_elements(By.CSS_SELECTOR, category_list_css)
        print(len(category_blocks))
        print(category_blocks[0].text)
        category_links = {element.get_attribute("innerHTML")[element.get_attribute("innerHTML").index("</i>")+4:].strip(): element.get_attribute('href')+"/au" for element in category_blocks}
        print(category_links)
        return category_links
    except Exception as e:
        print(e)
        signal = False
    finally:
        if driver:
            driver.close()
        clear_sessions()
        
    

category_list = get_categories_with_links(category_list_css, listing_website, chromedriver_command_executor) or {}
country = None
country_css = "div > div.agency-item--inner > div.agency-item--board > span:nth-child(1) > address *::text"
company_block_css = "#js-listing-app > ul > li.agency-list-item.agency-item"
company_name_css = "div > div.agency-item--inner > div.agency-item--main > div > div.info-head-wrap > div.info-head > h3 > a::text"
website_link_css = "div > div.agency-item--actions > a::attr(href)"
next_page_css = "#paginator > ul > li.page-item.active+li>a"
is_next_page_url_in_format = True

folder_name = "designrush_au"
# listing_website = "https://www.designrush.com/agency/digital-agencies"
# listing_website_start_url = "https://www.designrush.com/agency/web-development-companies/"
page_num = 1
# category = "Web Development"

51

{'Digital Agencies': 'https://www.designrush.com/agency/digital-agencies/au', 'Branding Agencies': 'https://www.designrush.com/agency/logo-branding/au', 'Creative Agencies': 'https://www.designrush.com/agency/creative-agencies/au', 'Product Design Companies': 'https://www.designrush.com/agency/product-design/au', 'Logo Design Companies': 'https://www.designrush.com/agency/logo-design/au', 'Graphic Design Companies': 'https://www.designrush.com/agency/graphic-design/au', 'Packaging Design Companies': 'https://www.designrush.com/agency/package-design/au', 'Video Production Companies': 'https://www.designrush.com/agency/video-production/au', 'Public Relations Firms': 'https://www.designrush.com/agency/public-relations/au', 'Design Agencies': 'https://www.designrush.com/agency/design-agencies/au', 'Reputation Management Companies': 'https://www.designrush.com/agency/reputation-management-companies/au', 'Web Design Companies': 'https://www.designrush.com/agency/website-design-developmen

In [6]:
# country_css = "div.provider-card__body.provider-body > ul > li:nth-child(3) > span > span::text"
# country = None
# company_block_css = ".directory-providers__list >li.provider-card"
# company_name_css = "div.provider-header > h3 > a::attr(title)"
# website_link_css = "div.provider-card__body.provider-body > div.provider-card__cta > a.provider-card__visit-btn.provider-visit.track-website-visit::attr(href)"
# next_page_css = "#directory-providers > div > nav > ul > li.page-item.next > a"
# listing_website_start_url = "https://themanifest.com/au/software-testing/companies"
# page_num = 901
# chromedriver_command_executor = "http://localhost:4444/wd/hub"
# folder_name = "themanifest_au"
# is_next_page_url_in_format = False

In [7]:
if not os.path.exists('data'):
    os.makedirs('data')
if not os.path.exists(f"data/{folder_name}"):
    os.makedirs(f"data/{folder_name}")
    os.makedirs(f"data/{folder_name}/emails")
    os.makedirs(f"data/{folder_name}/logs")
    os.makedirs(f"data/{folder_name}/out")

In [8]:
def run_crawl(
        full_path, page_num, category=None,
        country=country, 
        country_css=country_css,
        company_block_css=company_block_css, 
        company_name_css=company_name_css, 
        website_link_css=website_link_css, 
        folder_name=folder_name):
    subprocess.Popen(f'''cd {os.getcwd()} && \
                    scrapy crawl fetch_emails \
                    -a start_url="file://{full_path}" \
                    {f'-a country="{country}"' if country else ''} \
                    {f'-a country_css="{country_css}"' if country_css else ''} \
                    -a company_block_css="{company_block_css}" \
                    -a company_name_css="{company_name_css}" \
                    -a website_link_css="{website_link_css}" \
                    -a category="{category}" \
                    -o "data/{folder_name}/emails/{folder_name}{category}{country if country else ''}{page_num}.csv" \
                    --logfile "data/{folder_name}/logs/{folder_name}{category}{country if country else ''}{page_num}.log"''',
                    shell=True)

In [13]:
def get_companies(category_list, restart_category=None, restart_page_num=None, restart_next_page_url=None):
    # Set the options for the webdriver
    # PROXIES = get_proxies()
    options = Options()
    
    driver = None
    try:
        signal = True
        options.add_argument('--ignore-certificate-errors')
        options.add_argument('--ignore-ssl-errors=yes')
        options.add_argument('--no-sandbox')
        options.add_argument('--disable-dev-shm-usage')
        # options.add_argument("--headless")

        # Set the proxy
        # random_proxy = random.choice(PROXIES)
        # print("Proxy in use: %s" % random_proxy)
        # options.add_argument('--proxy-server=%s' % random_proxy)
        # options.add_argument('--proxy-server=%s' % '104.234.157.100:21278')

        driver = webdriver.Remote(
            command_executor=chromedriver_command_executor,
            options=options,
        )
        categories = list(category_list.keys())
        if restart_category:
            print(categories.index(restart_category))
            categories = categories[categories.index(restart_category):]
        
        for category in categories:
            listing_website_start_url = category_list[category]
            next_page = listing_website_start_url
            page_num = 1
            if restart_category:
                if category == restart_category:
                    if restart_next_page_url:
                        next_page = restart_next_page_url
                    page_num = restart_page_num
                else:
                    continue
            with concurrent.futures.ProcessPoolExecutor(max_workers=10) as executor:
                next_page_url = listing_website_start_url
                while next_page:
                    print(next_page)
                    driver.get(next_page)
                    print(driver.title)
                    if driver.title == "Just a moment...":
                        print("Just a moment...")
                        print("Waiting for 60 seconds")
                        time.sleep(60)
                        continue
                    file_path = f'data/{folder_name}/out/{folder_name} {category} {page_num}.html'
                    full_path = os.path.abspath(file_path)
                    with open(file_path, 'w') as f:
                        f.write(driver.page_source)
                    executor.submit(run_crawl, full_path, page_num, category)
                    if is_next_page_url_in_format:
                        next_page = next_page_url + f"?page={page_num+1}"
                    else:
                        next_page_element = driver.find_element(By.CSS_SELECTOR, next_page_css)
                        if next_page_element:
                            next_page = next_page_element.get_attribute('href')
                        else:
                            next_page = None
                        print(next_page)
                    page_num += 1
                    time.sleep(5)

    
    except NoSuchElementException as e:
        print("No Such Element Exception")
        print(e)
        if categories.index(category) == len(categories) - 1:
            signal = False
        signal = categories[categories.index(category)+1], page_num, next_page
    except WebDriverException as e:
        signal = (category,page_num, next_page)
        print(e)
    except Exception as e:
        print(e)
        print(e.__class__.__name__)
        signal = False
    finally:
        if driver:
            driver.close()
        clear_sessions()
        return signal


In [14]:
clear_sessions()
signal = get_companies(category_list=category_list, restart_category='Digital Agencies', restart_page_num=69, restart_next_page_url='https://www.designrush.com/agency/digital-agencies/au?page=69')
print(signal)
while isinstance(signal, tuple):
    signal = get_companies(category_list=category_list, restart_category=signal[0], restart_page_num=signal[1], restart_next_page_url=signal[2])
    print(signal)
    time.sleep(10)

0
https://www.designrush.com/agency/digital-agencies/au?page=69
Top 30 Digital Agencies in Australia - Feb 2024 Rankings | DesignRush
https://www.designrush.com/agency/digital-agencies/au?page=70
Top 30 Digital Agencies in Australia - Feb 2024 Rankings | DesignRush
https://www.designrush.com/agency/digital-agencies/au?page=71
Top 30 Digital Agencies in Australia - Feb 2024 Rankings | DesignRush
https://www.designrush.com/agency/digital-agencies/au?page=72
Top 30 Digital Agencies in Australia - Feb 2024 Rankings | DesignRush
https://www.designrush.com/agency/digital-agencies/au?page=73
Top 30 Digital Agencies in Australia - Feb 2024 Rankings | DesignRush
https://www.designrush.com/agency/digital-agencies/au?page=74
Top 30 Digital Agencies in Australia - Feb 2024 Rankings | DesignRush
https://www.designrush.com/agency/digital-agencies/au?page=75
Top 30 Digital Agencies in Australia - Feb 2024 Rankings | DesignRush
https://www.designrush.com/agency/digital-agencies/au?page=76
Top 30 Digit